In [2]:
from pathlib import Path
import numpy as np
import torch, torchvision, json
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
from efficientnet_pytorch import EfficientNet as EN
from tqdm import tqdm
device="cuda"

In [3]:
data_dir = Path('./data/tiny-imagenet-200')
data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
])

In [4]:
valid_set = torchvision.datasets.ImageFolder(data_dir / 'val', data_transforms)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=128,
                                           shuffle=True, num_workers=4, pin_memory=True)
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128,
                                           shuffle=True, num_workers=4, pin_memory=True)
loss = CrossEntropyLoss()

In [8]:
experiment = 6
seed = 0
model_id = f"{experiment}_{seed}"
model_path = Path(f"outputs/experiment_{experiment}/data/{model_id}").rglob('*pt')
model_path = list(model_path)[0]
model = EN.from_pretrained('efficientnet-b5', num_classes = 200)
model.load_state_dict(torch.load(model_path))
model.to(device)

Loaded pretrained weights for efficientnet-b5


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [9]:
def valid(model, dataset, dataloader, loss, device):
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader),
                            total=len(dataset) / dataloader.batch_size):
            data, y = data
            data = data.to(device)
            y = y.to(device)
            ans = model.forward(data)
            los = loss(ans, y)
            running_loss += los.item()
    return running_loss, running_loss / len(dataloader.dataset)

In [11]:
train_loss, avg_train = valid(model, train_set, train_loader, loss, device)
print(train_loss, avg_train)

  6%|████▌                                                                              | 433/7812.5 [01:15<21:21,  5.76it/s]


KeyboardInterrupt: 

In [10]:
valid_loss, avg_valid = valid(model, valid_set, valid_loader, loss, device)
print(valid_loss, avg_valid)

79it [00:14,  5.32it/s]                                                                                                      

596.2280378341675 0.05962280378341675
